In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install xformers --upgrade

import os
os.environ['TRITON_DISABLE_LINE_INFO'] = '1' # Optional, tested with and without
os.environ['TRITON_JIT_DISABLE_OPT'] = '1' # Likely the most critical change

Setting up and downloading pre trained model

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# Constants
person_model = "Alex oh"
pretrained_model = "unsloth/mistral-7b-bnb-4bit"

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = pretrained_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Adding Lora Adapter

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Parses a whatsapp style chat log into documents

In [ ]:
# data set parse
def parseLine(line):
    try:
        name = line[line.index(']') + 2: line.index(': ')]
        msg = line[line.index(': ') + 2:]
        return name, msg
    except:
        return None, line.strip()

class Document:
    def __init__(self, messages):
        self.messages = messages

    def to_dict(self):
        messages_dict = [{"from": name, "value": msg} for name, msg in self.messages]
        return messages_dict

def big_chunk():
    MAX = 8
    documents = []

    with open('_chat.txt', 'r') as file:
        prev_name = None
        prev_msg = ''

        curr_messages = []

        for line in file:
            name, msg = parseLine(line)
            msg = msg.replace('\n', ' ').strip()

            # Skip system messages
            if '\u200e' in msg or msg.strip() == "":
                continue

            # first messgage
            if prev_name is None:
                prev_name = name

            # continuation
            if name is None or name == prev_name:
                prev_msg += " " + msg

            # new name
            if name != prev_name:
                if prev_name == person_model:
                    curr_messages.append([prev_name, prev_msg])
                else:
                    curr_messages.append(['user', f'{prev_name}: {prev_msg}'])
                if len(curr_messages) >= MAX and prev_name == person_model:
                    document = Document(curr_messages)
                    curr_messages = []
                    documents.append(document)

                prev_name = name
                prev_msg = msg

        if len(curr_messages) > 0:
            documents.append(Document(curr_messages))

    return documents

documents = big_chunk()

# convert to json
documents = [document.to_dict() for document in documents]

In [ ]:
from unsloth.chat_templates import get_chat_template

unsloth_template = (
  "{{ bos_token }}"\
  "{{ 'You are Alex oh the assistant, maintain conversation as they would\\n' }}"\
  "{% for message in messages %}"\
      "{% if message['from'] == 'user' %}"\
          "{{ '>>> User: ' + message['value'] + '\\n' }}"\
      "{% elif message['from'] == 'Alex oh' %}"\ # Change this name to person_model
          "{{ '>>> Assistant: ' + message['value'] + '\\n' }}"\
      "{% endif %}"\
  "{% endfor %}"\
  "{% if add_generation_prompt %}"\
      "{{ '>>> Assistant: ' }}"\
  "{% endif %}"\
  "{{ eos_token }}"\
)

unsloth_eos_token = "eos_token"

tokenizer = get_chat_template(
    tokenizer,
    chat_template = (unsloth_template, "eos_token"),
    mapping = {
        "role": "from",
        "content": "value",
        "user": "user",    # Who provides the input
        "assistant": person_model     # Who gives the output to learn
    },
    map_eos_token = True
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]  # <- use batch of convos
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    print("Texts generated:", len(texts))
    return {"text": texts}


from datasets import load_dataset, Dataset
dataset = Dataset.from_list([{"conversations": doc} for doc in documents])

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names
)

# For testing of dataset
dataset[0]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = False,
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to='none',
    ),
)

trainer_stats = trainer.train()

Download for local use

In [ ]:
if True:
  from peft import PeftModel
  from transformers import AutoModelForCausalLM

  trainer.model.save_pretrained("outputs")
  tokenizer.save_pretrained("outputs")

  base_model = AutoModelForCausalLM.from_pretrained(pretrained_model)  # or your base model
  model = PeftModel.from_pretrained(base_model, "outputs")
  model = model.merge_and_unload()  # Merge LoRA into base model
  model.save_pretrained("merged_model")

  !zip -r outputs.zip merged_model/
  from google.colab import files
  files.download("outputs.zip")


Testing

In [ ]:
# Saves the model to be called for testing
if False:
  trainer.model.save_pretrained("outputs")
  tokenizer.save_pretrained("outputs")

  !zip -r outputs.zip outputs/
  from google.colab import files
  files.download("outputs.zip")



In [ ]:
from unsloth import FastLanguageModel
import torch

# Load the model using Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "outputs",  # Your saved model folder
    max_seq_length = 1024,   # Or whatever you used
    dtype = None,            # Or torch.float16 / bfloat16 if needed
    load_in_4bit = True,     # Or False depending on your setup
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

In [ ]:
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            eos_token_id=tokenizer.eos_token_id,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
# Test it
response = generate_response('''<s>You are Alex oh the assistant, maintain conversation as they would, reply as the assitant only
>>> User: shall we go to the beach tmr
>>> Assistant:''')
print(response)